In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, KFold
import pandas as pd
import numpy as np

In [1]:
from gensim.models.word2vec import Word2Vec
import pickle

D:\Python_storage\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### Загрузка данных

мы считываем предобработанные данные, которые прошли через функцию tokenizer_best

*данную функцию можно встретить в других ноутбуках моего репозитория*

In [6]:
# считываем токены трэйна
with open('token_train.pickle', 'rb') as dump_in:
    token_train = pickle.load(dump_in)

In [7]:
# считываем токены теста
with open('token_test.pickle', 'rb') as dump_in:
    token_test = pickle.load(dump_in)

In [8]:
sentences = token_train + token_test

In [9]:
# Считываем обучающую и тестовую выборки
train = pd.read_csv('train_data.csv', encoding='utf', engine='python', index_col=0)
test = pd.read_csv('test_data.csv', encoding='utf', engine='python', index_col=0)

### Обучение модели w2v и извлечение признаков из неё

In [34]:
%%time
model = Word2Vec(size=300, min_count=1)
model.build_vocab(sentences)

Wall time: 1.9 s


In [35]:
%%time
model.train(sentences,total_examples=len(sentences),epochs=3000)


Wall time: 1h 2min 19s


(2862194889, 3128013000)

In [36]:
model.wv.most_similar('кредит') 

[('автокредит', 0.5961386561393738),
 ('ипотека', 0.569599986076355),
 ('кредитный', 0.3977956175804138),
 ('рассрочка', 0.38397979736328125),
 ('кредитка', 0.38149750232696533),
 ('карта', 0.3735683560371399),
 ('сумма', 0.3481549620628357),
 ('досрочно', 0.347896933555603),
 ('потребительский', 0.34564313292503357),
 ('банк', 0.33431875705718994)]

In [37]:
import pandas
def SentenceToAverageWeightedVector(wv, sentence):
    """Функция для извлечения признаков из обученной модели W2V"""
    vectors = pandas.DataFrame()
    index = 0
    try:
        for word in sentence:
            if word in wv.vocab:
                vectors[index] = wv[word]
            index += 1
        vectors = vectors.transpose()
        vector = vectors.mean().values.tolist()
    except Exception:
        return []
    return vector

In [38]:
%%time
vectors_merged_train = [SentenceToAverageWeightedVector(model.wv, senten) for senten in token_train]

Wall time: 4min 18s


In [39]:
%%time
vectors_merged_test = [SentenceToAverageWeightedVector(model.wv, senten) for senten in token_test]

Wall time: 1min 4s


### Модель Логистической регрессии

In [40]:
y1 = pd.factorize(train.type)[0]

In [41]:
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')

In [42]:
%%time
pred_scores = cross_val_score(estimator=clf, X=vectors_merged_train, y=y1,
                scoring='accuracy',  
                cv=10, #stratified by default
                n_jobs=-1)
print(np.mean(pred_scores))

0.6585077635255752
Wall time: 2min 2s


### Выполним сеточный поиск параметров

In [45]:
lm = LogisticRegression(#solver='newton-cg', 
                        n_jobs=-1,
                        solver='lbfgs',
                        penalty='l2',
                        tol=0.000000001,
                        random_state=42,
                        C=10, 
                        max_iter=100000)
lm_params = {#'penalty':[ 'l2'], #'l1',
             'C':[0.001, 0.01, 0.1, 1, 2, 5, 10, 20, 100],
             'solver':['lbfgs'],
             'tol' : [10, 1, 0.1, 0.01, 0.001, 0.0001, 0.0001]
}
lm_search = GridSearchCV(estimator=lm, 
                         param_grid=lm_params, 
                         scoring ='accuracy', 
                         cv=StratifiedKFold(10), 
                         n_jobs=-1,
                        )

In [46]:
%%time
lm_search_fitted = lm_search.fit(X=vectors_merged_train, y=y1)
print(lm_search_fitted.best_score_)

D:\Users\anaconda_py\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6722762539613157
Wall time: 1h 15min 46s


__Увы__, результаты получились очень низкими. Возможно, стоило попробовать как-то иначе извлекать и обрабатывать токены, либо, использовать дополнительные линейные модели.